# Integrating Refuel Autolabel into EvaDB

## 1. Setup

In [1]:
!pip install 'refuel-autolabel'
!pip install evadb

  Obtaining dependency information for refuel-autolabel from https://files.pythonhosted.org/packages/5c/6c/08eb43748e8b9a4af05dd10346c852f3cfb8feeb1dc66407794d2350cbf9/refuel_autolabel-0.0.16-py3-none-any.whl.metadata
  Using cached refuel_autolabel-0.0.16-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for loguru>=0.5.0 from https://files.pythonhosted.org/packages/03/0a/4f6fed21aa246c6b49b561ca55facacc2a44b87d65b8b92362a8e99ba202/loguru-0.7.2-py3-none-any.whl.metadata
  Using cached loguru-0.7.2-py3-none-any.whl.metadata (23 kB)
  Obtaining dependency information for numpy>=1.23.0 from https://files.pythonhosted.org/packages/2e/54/218ce51bb571a70975f223671b2a86aa951e83abfd2a416a3d540f35115c/numpy-1.26.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.0 MB/s eta 0:00:00
  Obtaining dependency information for requests>=2.27.0 from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a2

In [2]:
import os
import evadb
from my_secrets import OPENAI_KEY
# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [3]:
cursor = evadb.connect().cursor()
print("Connected to EvaDB")

Connected to EvaDB


In [4]:
from autolabel import get_data
get_data('walmart_amazon')

## 2. RefuelAutolabel Function

### 2.1 Create Function in EvaDB

I implemented the function which supports autolabeling via Refuel in functions/refuel_autolabel.py. The cell below simply creates the function in EvaDB using my implementation

I'm supporting the following actions:

- plan - plan the autolabeling task by giving a cost estimate and providing an example prompt which will be used to label an example
- run - run the autolabeling task and insert the generated labels into the dataset
- explain - explain the label generated for each example in the dataset

In Refuel, these actions are all implemented in the LabelingAgent class. This class is the one we use to label datasets if we use Refuel's Python API

In [7]:
create_function_query = f"""CREATE FUNCTION IF NOT EXISTS RefuelAutolabel
            IMPL  '../functions/refuel_autolabel.py';
            """
cursor.query("DROP FUNCTION IF EXISTS RefuelAutolabel;").execute()
cursor.query(create_function_query).execute()
print("Created Function")

11-25-2023 21:11:09 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_function:0089] Function RefuelAutolabel does not exist, therefore cannot be dropped.
2023-11-25 21:11:09 evadb.utils.logging_manager WARNING: Function RefuelAutolabel does not exist, therefore cannot be dropped.


Created Function


### 2.2 Plan Autolabeling

The following cell uses the autolabeling configuration specified by config_banking.json and the size of seed.csv to estimate the cost of labeling seed.csv. Furthermore, it gives an example of a prompt that would be generated and given to ChatGPT to perform the given autolabeling task

The prompt asks the LLM to classify a complaint into one of several predetermined categories and gives the LLM several examples with which to learn from

In [8]:
query= f""" SELECT RefuelAutolabel("plan", '../configs/config_banking.json', 'seed.csv');"""
result = cursor.query(query).execute()

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $0.5985 │
│ Number of Examples       │ 199     │
│ Average cost per example │ $0.003  │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at understanding bank customers support complaints and queries.
Your job is to correctly classify the provided input example into one of the following categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchange_charge
exchange_rate
exchange_via_app
extra_charge_on_statement
failed_transfer
fiat_currency_support
get_disposable_virtual_card
get_physical_card
getting_spare_card
getting_virtual_card
lost_or_stolen_card
lost_or_stolen_phone
order_physical_card
passcode_forgotten
pending_card_payment
pending_cash_withdrawal
pending_top_up
pending_transfer
pin_blocked
receiving_money
Refund_not_showing_up
request_refund
reverted_card_payment?
supported_cards_and_currencies
terminate_account
top_up_by_bank_transfer_charge
top_up_by_card_charge
top_up_by_cash_or_cheque
top_up_failed
top_up_limits
top_up_reverted
topping_up_by_card
transaction_charged_twice
transfer_fee_charged
transfer_into_account
transfer_not_received_by_recipient
transfer_timing
unable_to_verify_identity
verify_my_identity
verify_source_of_funds
verify_top_up
virtual_card_not_working
visa_or_mastercard
why_verify_identity
wrong_amount_of_cash_received
wrong_exchange_rate_for_cash_withdrawal

You will answer with just the the correct output label and nothing else.

Some examples with their output answers are provided below:

Input: I have withdrawn cash from ATM but i received the wrong amount. I want cash back as in app its showing 
actual amount which i got. Please help me in this.
Output: wrong_amount_of_cash_received
Input: The app made a mistake and said I made a cash withdrawal.
Output: cash_withdrawal_not_recognised
Input: I tried to take out cash but the amount isn't right, so what do I do?
Output: wrong_amount_of_cash_received
Input: I received cash from the ATM earlier, but this shows up as pending in the app.  I've got the cash already.  
How can this be still pending?
Output: pending_cash_withdrawal
Input: did not receive correct cash upon withdrawal
Output: wrong_amount_of_cash_received
Input: I didn't receive the right amount of cash back
Output: wrong_amount_of_cash_received
Input: I took out money from a transaction machine and it exchanged the wrong dollar value amount from another 
currency!
Output: wrong_exchange_rate_for_cash_withdrawal
Input: Why did the ATM machine fail to give me a cash withdrawal?
Output: declined_cash_withdrawal
Input: I checked the exchange rate when I withdrew cash. But the actual rate that you applied was different.
Output: wrong_exchange_rate_for_cash_withdrawal
Input: How can I withdraw money?
Output: atm_support

Now I want you to label the following example:
Input: I have withdrawn cash from ATM but i received the wrong amount. I want cash back as in app its showing 
actual amount which i got. Please help me in this.
Output:

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

### 2.3 Run Autolabeling

Simple example with no optional arguments passed

In [9]:
"""
use the function arguments to specify
    1. the mode to execute in (plan, run, explain)
    2. config file
    3. dataset file
    4. (optional) output file
    5. (optional) max items
    6. (optional) start index
    7. (optional) skip eval
"""

query_with_output = f""" SELECT RefuelAutolabel("run", '../configs/config_banking.json', 'seed.csv', 'output.csv');"""

query= f""" SELECT RefuelAutolabel("run", '../configs/config_banking.json', 'seed.csv');"""
result = cursor.query(query).execute()

Output()

Actual Cost: 0.0

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.9648   │ 199     │ 1.0             │
└──────────┴─────────┴─────────────────┘

Using all of the optional arguments to label 10 examples starting at the 1st example

In [10]:
query= f""" SELECT RefuelAutolabel("run", '../configs/config_walmart.json', 'seed.csv', 'output.csv', '10', '1', 'false');"""
result = cursor.query(query).execute()

Output()

Actual Cost: 0.0006

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.0      │ 10      │ 1.0             │
└──────────┴─────────┴─────────────────┘

### 2.4 Explain Autolabeling

Although the following cell threw an error, it did generate an explanation for all the examples and labels in seed.csv

In [13]:
query= f"""SELECT RefuelAutolabel("explain", '../configs/config_banking.json', 'seed.csv');"""
result = cursor.query(query).execute()

Output()

2023-11-25 21:27:02 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=30.0).


11-25-2023 21:32:38 ERROR [plan_executor:plan_executor.py:execute_plan:0182] Batch constructor not properly called.
Expected pandas.DataFrame, got <class 'list'>
Traceback (most recent call last):
  File "/Users/krishnathan/Dropbox (GaTech)/Documents/College/Fall 23/CS 6422/Project/evadb-project-2/evadb_env/lib/python3.11/site-packages/evadb/executor/plan_executor.py", line 178, in execute_plan
    yield from output
  File "/Users/krishnathan/Dropbox (GaTech)/Documents/College/Fall 23/CS 6422/Project/evadb-project-2/evadb_env/lib/python3.11/site-packages/evadb/executor/project_executor.py", line 42, in exec
    batch = apply_project(dummy_batch, self.target_list)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/krishnathan/Dropbox (GaTech)/Documents/College/Fall 23/CS 6422/Project/evadb-project-2/evadb_env/lib/python3.11/site-packages/evadb/executor/executor_utils.py", line 69, in apply_project
    batches = [expr.evaluate(batch) for expr in project_list]
       

ExecutorError: Batch constructor not properly called.
Expected pandas.DataFrame, got <class 'list'>

### 2.5 Bringing the labeled dataset into EvaDB

Here, I'm pulling the labeled banking dataset into EvaDB. I had to rename it to seed_banking.csv and test_banking.csv so that I could run the Walmart example with only labelling 10 examples. I'm not sure why the dataset currently being labeled has to be named seed.csv and test.csv. Unfortunately, Refuel's API does not allow me to specify the dataset names in a more flexible way. This could be something to investigate further if we wish to work with several datasets

In [75]:
drop_query = "DROP TABLE IF EXISTS MyCSV"
cursor.query(drop_query).execute()

query1 = """CREATE TABLE IF NOT EXISTS MyCSV (
                example TEXT(1000),
                label TEXT(100),
                explanation TEXT(1000)
            );"""

query2 = "LOAD CSV 'test_banking.csv' INTO MyCSV;"
query3 = "SELECT * FROM MyCSV;"

result = cursor.query(query1).execute()
result = cursor.query(query2).execute()
result = cursor.query(query3).execute()

print(result)

      _row_id  \
0           1   
1           2   
2           3   
3           4   
4           5   
...       ...   
1993     1994   
1994     1995   
1995     1996   
1996     1997   
1997     1998   

                                                                                                  example  \
0                                                                              I want to close my account   
1     It seems I was overcharged when I used an ATM while on vacation. If I knew about your fees in ad...   
2                                I have a direct debit transaction I have not set up, but would like to .   
3                                                         How much does it cost in fees to use your card?   
4                                                                             There is an extra $1 charge   
...                                                                                                   ...   
1993                            